In [2]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install rouge_score

In [3]:
from datasets import load_dataset

ds = load_dataset("ninadn/indian-legal")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/210M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/3.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7030 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [4]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
import nltk
nltk.download('punkt')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
max_input = 512
max_target = 128
batch_size = 3
model_checkpoints = "facebook/bart-large-xsum"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [6]:
def preprocess_data(data_to_process):
    inputs = data_to_process['Text']
    targets = data_to_process['Summary']
    inputs = [" " if text is None else text for text in inputs]
    targets = [" " if text is None else text for text in targets]
    model_inputs = tokenizer(inputs, max_length=max_input, padding='max_length', truncation=True)
    targets_tokenized = tokenizer(text_target=targets, max_length=max_target, padding='max_length', truncation=True)
    model_inputs['labels'] = targets_tokenized['input_ids']
    return model_inputs
tokenize_data = ds.map(preprocess_data, batched=True)

Map:   0%|          | 0/7030 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [7]:
num_valid = 200
train_valid_split = tokenize_data['train'].train_test_split(test_size=num_valid, shuffle=True, seed=42)

# Rename keys for clarity
tokenize_data['train'] = train_valid_split['train']
tokenize_data['valid'] = train_valid_split['test']

# Print sizes to confirm
print(f"Train dataset size: {len(tokenize_data['train'])}")
print(f"Validation dataset size: {len(tokenize_data['valid'])}")
print(f"Text dataset size: {len(tokenize_data['test'])}")

Train dataset size: 6830
Validation dataset size: 200
Text dataset size: 100


In [8]:
tokenize_data

DatasetDict({
    train: Dataset({
        features: ['Text', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6830
    })
    test: Dataset({
        features: ['Text', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['Text', 'Summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})

In [9]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [10]:
batch_size = 3

In [11]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
import evaluate
metric = evaluate.load("rouge")

In [14]:
metric = evaluate.load("rouge")

In [20]:
def compute_rouge(pred):
  predictions, labels = pred
  #decode the predictions
  decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  #decode labels
  decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  #compute results
  res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
  #get %
  res = {key: value * 100 for key, value in res.items()}

  pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  res['gen_len'] = np.mean(pred_lens)

  return {k: round(v, 4) for k, v in res.items()}

In [26]:
args = transformers.Seq2SeqTrainingArguments(
    'legal-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size= 1,
    per_device_eval_batch_size= 1,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=1,
    fp16=True
    )

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data['train'],
    eval_dataset=tokenize_data['valid'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

<ipython-input-27-ffa2eda97ea9>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Seq2SeqTrainer(


In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.943900,1.931821,36.107600,17.937200,26.357700,32.060900,62.000000
2,2.631000,1.912372,36.416200,18.272700,26.569000,32.434600,61.930000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.943900,1.931821,36.107600,17.937200,26.357700,32.060900,62.000000
2,2.631000,1.912372,36.416200,18.272700,26.569000,32.434600,61.930000
3,2.207700,1.958491,36.737400,18.007400,26.638000,32.502000,62.000000


TrainOutput(global_step=10245, training_loss=2.644367804464449, metrics={'train_runtime': 6778.2893, 'train_samples_per_second': 3.023, 'train_steps_per_second': 1.511, 'total_flos': 2.220198664863744e+16, 'train_loss': 2.644367804464449, 'epoch': 3.0})